# DDL

- data definition language
- specify information about relations, including
    - the schema for each relation
    - the domain of values associated with each attribute
    - integrity constraints
    - set of indexes to be maintained for each relation
    - physical storage structure of each relation (e.g., choice of InnoDB vs MyISAM storage engine in MySQL)

## Domain Types

- **char(n)**
    - Fixed length character string, with length $n$
- **varchar(n)**
    - Variable length character string, with maximum length $n$
- **int**
    - integer, machine-dependent finite subset of the integers
- **smallint**
    - small integer
- **numeric(p, d)**
    - fixed point number, with precision of $p$ significant digits, with $d$ digits to the right of decimal point
- **real, double precision**
    - floating point number and double-precision floating point number with machine-dependent precision
- **float(n)**
    - floating point number with precision of at least $n$
- **date, time**
    - calendar date (YYYY-MM-DD), and time of day (hh:mm:ss)

## Create Table Construct

- **create table**: define an SQL relation
```SQL
create table r(
    A_1 D_1, 
    ...,
    A_n D_n,
    (integrity-constaint 1),
    ...,
    (integrity-constaint k),
```
- $r$: relation
- $A_i$: attribute
- $D_i$: domain of $A_i$

**Example**

```SQL
create table instructor(
    ID        char(5),
    name      varchar(20),
    dept_name varchar(20),
    salary    numeric(8, 2)
)
```

## Drop and Alter Table Constructs

- **drop table**
    - deletes the table and its contents, assumes table exists
    - `drop table student`
- **drop table if exists**
    - deletes the table and its contents if table exists
    - `drop table if exists student`
- **delete from**
    - deletes all tuples from table, but retains table
    - `delete from student`
- **alter table**
    - **alter table $r$ add $A$ $D$**
        - adds attribute with name $A$ and domain $D$ to relation $r$, all existing tuples in the relation are assigned ***null*** as the value for the new attribute
    - **alter table $r$ drop A**
        - drops attribute $A$ from relation $r$


## Declaring Integrity Constraints

- **not null**
- **primary key ($A_1$, ..., $A_n$)** ensures uniqueness
    - at most once per table
    - primary key attribute cannot be null
    - the **primary key** constraint on an attribute implies the **not null** constraint
    - order of keys does matter and it concerns performance
- **unique ($A_1$, ..., $A_n$)** ensures uniqueness (superkey)
    - can have more than one per table
    - **primary key** and **unique** both identify superkeys
    - the number of tuples in a **unique** attribute that are allowed to be nullable is system-dependent (0, 1, or many)
- **foreign key ($A_m$, ..., $A_n$) references $r$($A'_m$, ..., $A'_n$)**
    - defines a foreign key in the child (referencing) table that points to a referenced key in a parent (referenced) table $r$
    - **referential integrity**
    - in MySQL the referenced key must be a superkey of the referenced table, or a prefix of a multi-attribute primary key of the referenced table
    - caveats: MySQL will enforce foreign key constraints only in some cases
        - the storage engine for the parent and child tables must support foreign keys (e.g., InnoDB)
        - correpsonding attributes must have "similar internal data types"
        - foreign key and referenced key **must be indexed**
- **default $V$**: makes $V$ the default value for an attribute

```SQL
create table instructor(
    ID        char(5),
    name      varchar(20) not null,
    dept_name varchar(20),
    salary    numeric(8, 2) default 0,
    primary key (ID),
    foreign key (dept_name) references department(dept_name)
)
```

## Referential Actions

- define the behavior of the DB in cases when an update or deletion on the parent (i.e., referenced) table SQL statement affects a value referenced by a child (i.e., referencing) table
- **cascade**: automatically update or delete foreign key in matching rows of child table
- **set null**: set foreign key columns to *null* in matching rows of child table (assumes foreign key is nullable)
- **set default**: set foreign key columns to the default value in matching rows of child table
- **no action**: reject operation and generate error
    - in MySQL it is called *strict*

```SQL
create table instructor (
    ...,
    dept_name varchar(20),
    foreign key (dept_name) references department(dept_name)
    on update cascade on delete set null
)
```


## Check Constraints

- in SQL 92, constraints can be expressed using predicates and declared using the **check** clause

```SQL
create table instructor(
    ...,
    dept_name varchar(20),
    salary    numeric(8, 2),
    foreign key (dept_name) references department(dept_name),
    check (salary > 50000 and salary < 150000)
)
```

# DML

## Modifying Relations

### Insertion

```SQL
insert into course
    values ('ECE-356', 'Databases', 'ECE', 0.5)
```

### Updates

```SQL
-- give a 3% salary increase to all instructors whose salary is below 80,000
update instructor
    set salary = salary * 1.03
    where salary < 80000
```

```SQL
-- give a 3% salary increase to all instructors whose salary < 80000
-- and a 2% salary increase to all instructors whose salary is >= 80000
update instructor
    set salary = salary *
        (case
             when salary < 80000 then 1.03
             when salary >= 80000 then 1.02
         end)
```
- note: if the set of cases defined is not exhasutive, the case statement returns null, and else case can be included to prevent this

### Deletion
```SQL
-- delete all instructors
delete from instructor
```

```SQL
delete from instructor
where dept_name = 'Math'
```

```SQL
delete from instructor
where dept_name in (select dept_name
                    from department
                    where building = 'EIT')
```

## String Pattern Matching

- **like** uses patterns that are described using two special characters (wildcards)
    - `%`: matches any substring
    - `_`: matches any one character

```SQL
-- find the names of all instructors whose name includes the substring "dar"

select name from instructor where name like '%dar%'

-- match the string '100 %'

... like '100 \%' escape '\'
```



## Limiting Output

- **limit** takes the first N records from the result

```SQL
-- find the three highest-paid instructors in the ECE department assuming that salaries are distinct

select name from instructor where dept_name = 'ECE' order by salary desc limit 3
```

- this technique does not answer "top-k" queries correctly in the special case when ties are possible

## Joins (more)

- **Join operations** takes two relation and return another relation as a rsult
- **Join condition** defines which tuples in the two relations match, and what attributes are present in the result of the join
    - `natural`
    - `on <predicate>`
    - `using (A1, A2, ..., An)`
- **Join type** defines how tuples in each relation that do not match any tuple in the other relation are treated (based on the join condition)
    - inner join
    - left outer join
    - right outer join
    - full outer join
- remark: `using` vs `on`
    - in cases of `JOIN ... ON`, e.g., `select * from t1 join t2 on t1.col1 = t2.col1`, then `col1` will appear twice in the result set
    - in cases of `USING`, e.g., `select * from t1 join t2 using (col1)`, then `col1` appears only once

### Inner and Outer Joins

Course             |  Prereq
:-------------------------:|:-------------------------:
<img src="img/Snip20190923_126.png"/>  |  <img src="img/Snip20190923_127.png"/>

- Inner Join
```SQL
SELECT * FROM course INNER JOIN prereq ON course.course_id = prereq.course_id
```

<img src="img/Snip20190923_128.png" width=60%/>

- Left Outer Join
```SQL
SELECT * FROM course LEFT OUTER JOIN prereq ON course.course_id = prereq.course_id
```

<img src="img/Snip20190923_129.png" width=60%/>

- Right Natural Outer Join
```SQL
SELECT * FROM course NATURAL RIGHT OUTER JOIN prereq
```

<img src="img/Snip20190923_130.png" width=60%/>

- Full Outer Join
```SQL
SELECT * FROM course FULL OUTER JOIN prereq USING (course_id)
```

<img src="img/Snip20190923_131.png" width=60%/>


# Views

- in some cases, it is not desirable for all users to see all the relations stored in a database instance
- a **view** provides a mechanism to hide certain data from the view of certain users
- any relation that is not part of the conceptual model but is made visable to a user as a "virtual relation" is called a **view**

- **Create view**: define a view
    - `CREATE VIEW view_name as <query expression>`
    - query expression is any legal SQL query
- once a view is defined, the view name can be used to refer to the virtual relation
- **a view can be defined over the result set of a join or group by query, and does not need to be defined over a single table**
- views are *dynamic*, meaning that **changing the data in the relations referenced by a view causes analogous changes in the virtual relation** corresponding to the view

**Examples**
- a view of instructors without the salary
```SQL
CREATE VIEW faculty AS
    SELECT ID, name, dept_name
    FROM instructor
```

- create a view of department salary totals
```SQL
CREATE VIEW departments_total_salary(dept_name, total_salary) AS
    SELECT dept_name, sum(salary)
    FROM instructor
    GROUP BY dept_name
```

- query from the above view
```SQL
SELECT * FROM departments_total_salary
```

# Auto-increment Attributes

- can be used to automatically generate primary key values

```SQL
CREATE TABLE instructor_auto(
    ID        int auto_increment primary key,
    name      varchar(20),
    dept_name varchar(20),
    salary    numeric(8, 2)
)

INSERT INTO instructor_auto(name, dept_name, salary)
VALUES ('Watson', 'Biology', 90210)

SELECT * FROM instructor_auto
```

- `ID=1` should be assigned automatically to the first tuple
- the auto-increment type **does not exist** in the ER model and therefore it does not eliminate the need for weak entity sets

# Stored Procedures

- a subroutine that consolidates and centralizes logic that would otherwise be implemented in database applications
- reduces redundant SQL code in applications

**Example**
```SQL
DELIMITER @@
CREATE PROCEDURE ProcTopSalary (IN dept_name VARCHAR(20))
BEGIN
    SELECT max(salary) FROM instructor
    WHERE instructor.dept_name = dept_name;
END @@
DELIMITER;

CALL ProcTopSalary('Physics');
```

- stored procedures are generally long-lived, it can be deleted using the drop keyword
    - `DROP PROCEDURE ProcTopSalary`

# Cursors

- a database **cursor** is a control structure that enables traversal of records in a database
- used inside stored procedures

```SQL
DELIMITER PROCEDURE CursorDemo()
BEGIN
    DECLARE year INT DEFAULT 2016;
    DECLARE my_id VARCHAR(5);
    DECLARE my_dept_name VARCHAR(20);
    DECLARE done INT DEFAULT 0;
    DECLARE cur CURSOR FOR
        SELECT ID, dept_name FROM instructor
    DECLARE CONTINUE HANDLER FOR
        NOT FOUND SET done = 1;
    OPEN cur;
    my_loop: LOOP
        FETCH cur INTO my_id, my_dept_name;
        IF done = 1 THEN
            LEAVE my_loop;
        END IF;
        IF my_dept_name = 'Biology' THEN
            INSERT INTO teaches
            VALUES (my_id, 'BIO-101', 1, 'Summer', year);
            SET year = year + 1;
        END IF;
    END LOOP my_loop;
    CLOSE cur;
END @@
DELIMITER;
```

# Triggers

- a **trigger** is procedural code that is automatically executed in response to certain events on a particular table or view in a database
- MySQL supports three types of trigger events: insert, update, and delete, a trigger can be executed either before or after the event

```SQL
DELIMITER @@
CREATE TRIGGER SalaryTrigger
BEFORE UPDATE ON instructor FOR EACH ROW
BEGIN
    IF NEW.salary > OLD.salary * 1.10 THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'increase higher than 10%';
    END IF;
END; @@
DELIMITER ;
```